In [11]:
#import modules and packages

import pandas as pd #data processing
import numpy as np #maths
import requests as req #get data from url
import gzip
from io import BytesIO

import seaborn as sns
sns.set()


In [1]:
#get film datasets

#set urls
url_title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz' #film name, year, runtime, genres
url_crew = 'https://datasets.imdbws.com/title.principals.tsv.gz'#actors, actresses, cinematographers, directors (redundant)
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz' #ratings for films (not all)
url_names = 'https://datasets.imdbws.com/name.basics.tsv.gz' #link table for names against nconst
url_langs = 'https://datasets.imdbws.com/title.akas.tsv.gz' #link table for names against nconst


#download from url
res_title_basics = req.get(url_title_basics).content
res_crew = req.get(url_crew).content
res_ratings = req.get(url_ratings).content
res_names = req.get(url_names).content
res_lang = req.get(url_langs).content


#decompress
title_basics_gzip = gzip.decompress(res_title_basics)
crew_basics_gzip = gzip.decompress(res_crew)
title_ratings_gzip = gzip.decompress(res_ratings)
names_gzip = gzip.decompress(res_names)
title_langs_gzip = gzip.decompress(res_lang)


#read csv into dataframes
titles = pd.read_csv(BytesIO(title_basics_gzip), delimiter='\t')
crew = pd.read_csv(BytesIO(crew_basics_gzip), delimiter='\t')
ratings = pd.read_csv(BytesIO(title_ratings_gzip), delimiter='\t')
names = pd.read_csv(BytesIO(names_gzip), delimiter='\t')
langs = pd.read_csv(BytesIO(title_langs_gzip), delimiter='\t')



NameError: name 'req' is not defined

In [ ]:
#clean data

#filter only English-speaking regions
desired_regions = ['CA', 'US', 'GB', 'IE', 'AU', 'NZ']
filtered_langs = langs[langs['region'].isin(desired_regions)]
tconsts_filtered_langs = filtered_langs['titleId'].tolist()


#remove non-movie rows, and movies from before 1960
titles = titles[titles['titleType'] == 'movie']
titles = titles[titles['startYear'] >= '1955']
titles = titles[titles['tconst'].isin(tconsts_filtered_langs)]

#get tconsts for remaining non-movie rows
film_tconsts = titles['tconst'].tolist()

#remove corresponding non-movie rows
crew = crew[crew['tconst'].isin(film_tconsts)]
ratings = ratings[ratings['tconst'].isin(film_tconsts)]

#set columns to remove from dataset
remove_from_titles = ['originalTitle', 'endYear', 'titleType', 'isAdult']
remove_from_crew = ['ordering','job','characters']
remove_from_ratings = ['numVotes']
remove_from_names = ['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']

titles = titles.drop(columns=remove_from_titles)
crew = crew.drop(columns=remove_from_crew)
ratings = ratings.drop(columns=remove_from_ratings)
names = names.drop(columns=remove_from_names)


In [ ]:
titles

,tconst,primaryTitle,startYear,runtimeMinutes,genres
27745,tt0028248,Shipmates o' Mine,2022,87,Musical
34798,tt0035423,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance"
35961,tt0036606,"Another Time, Another Place",1983,118,"Drama,War"
38005,tt0038687,Let There Be Light,1980,58,"Documentary,War"
38753,tt0039442,"Habla, mudita",1973,88,Drama
...,...,...,...,...,...
10436940,tt9916190,Safeguard,2020,95,"Action,Adventure,Thriller"
10436979,tt9916270,Il talento del calabrone,2020,84,Thriller
10437024,tt9916362,Coven,2020,92,"Drama,History"
10437056,tt9916428,The Secret of China,2019,\N,"Adventure,History,War"


In [ ]:
crew

,tconst,nconst,category
219758,tt0028248,nm0526951,actor
219759,tt0028248,nm0308249,actor
219760,tt0028248,nm0012499,actress
219761,tt0028248,nm0088942,actor
219762,tt0028248,nm0228839,actor
...,...,...,...
59836715,tt9916428,nm0422639,actress
59836716,tt9916428,nm0910951,director
59836717,tt9916428,nm8680851,actor
59836718,tt9916428,nm3370295,actor


In [ ]:
names

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
13133608,nm9993714,Romeo del Rosario
13133609,nm9993716,Essias Loberg
13133610,nm9993717,Harikrishnan Rajan
13133611,nm9993718,Aayush Nair


In [ ]:
ratings

,tconst,averageRating
12165,tt0028248,4.2
18023,tt0035423,6.4
19021,tt0036606,6.4
20718,tt0038687,7.4
21351,tt0039442,6.1
...,...,...
1384627,tt9916170,7.0
1384628,tt9916190,3.7
1384635,tt9916270,5.8
1384639,tt9916362,6.4


In [ ]:
#merge datasets for one complete table

crew_data = crew.pivot_table(index='tconst', columns='category', values='nconst', aggfunc=lambda x: ', '.join(x)).reset_index()
crew_data = crew_data.drop(columns=['archive_footage','archive_sound','self'])
film_data = pd.merge(titles, ratings, on='tconst', how='left')
film_data = pd.merge(film_data, crew_data, on='tconst', how='left')

film_data

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,actor,actress,cinematographer,composer,director,editor,producer,production_designer,writer
0,tt0028248,Shipmates o' Mine,2022,87,Musical,4.2,"nm0526951, nm0308249, nm0088942, nm0228839, nm...",nm0012499,NaN,NaN,nm0593632,NaN,NaN,NaN,"nm0068760, nm0669260"
1,tt0035423,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance",6.4,"nm0413168, nm0000630, nm0005227",nm0000212,nm0238698,nm0448843,nm0003506,nm0107463,nm0465298,NaN,nm0737216
2,tt0036606,"Another Time, Another Place",1983,118,"Drama,War",6.4,"nm0561155, nm0269416, nm0743027",nm0517642,nm0005683,nm0572820,nm0705535,NaN,nm0675294,NaN,"nm0450407, nm0485392"
3,tt0038687,Let There Be Light,1980,58,"Documentary,War",7.4,nm0404158,NaN,NaN,NaN,nm0001379,NaN,NaN,NaN,nm0442105
4,tt0039442,"Habla, mudita",1973,88,Drama,6.1,"nm0007023, nm0019330","nm0544330, nm0370455",nm0190834,nm0006280,nm0349426,nm0215327,NaN,nm0651204,nm0305869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326308,tt9916190,Safeguard,2020,95,"Action,Adventure,Thriller",3.7,"nm0302466, nm5464699, nm2018573, nm2009056",NaN,"nm8262250, nm5785503",nm7879007,nm7308376,nm4877791,nm10299811,NaN,NaN
326309,tt9916270,Il talento del calabrone,2020,84,Thriller,5.8,"nm0144812, nm3080119","nm2063290, nm9428255",nm0130846,nm2747888,nm1480867,NaN,"nm1799384, nm0656465",NaN,nm10538402
326310,tt9916362,Coven,2020,92,"Drama,History",6.4,"nm0107165, nm0266723","nm3766704, nm10678594",NaN,nm5813626,nm1893148,NaN,"nm2970042, nm4065853, nm1086949",NaN,nm3471432
326311,tt9916428,The Secret of China,2019,\N,"Adventure,History,War",3.5,"nm3611859, nm9445072, nm8594703, nm8680851, nm...",nm0422639,NaN,NaN,nm0910951,NaN,NaN,NaN,NaN


In [ ]:
#remove movies that don't have enough data (roughly 70,000)

rows_with_no_info = film_data[film_data[['actor', 'actress', 'cinematographer', 'producer', 'editor', 'composer', 'averageRating']].isna().all(axis=1)]
film_data = film_data.drop(rows_with_no_info.index)

film_data

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,actor,actress,cinematographer,composer,director,editor,producer,production_designer,writer
0,tt0028248,Shipmates o' Mine,2022,87,Musical,4.2,"nm0526951, nm0308249, nm0088942, nm0228839, nm...",nm0012499,NaN,NaN,nm0593632,NaN,NaN,NaN,"nm0068760, nm0669260"
1,tt0035423,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance",6.4,"nm0413168, nm0000630, nm0005227",nm0000212,nm0238698,nm0448843,nm0003506,nm0107463,nm0465298,NaN,nm0737216
2,tt0036606,"Another Time, Another Place",1983,118,"Drama,War",6.4,"nm0561155, nm0269416, nm0743027",nm0517642,nm0005683,nm0572820,nm0705535,NaN,nm0675294,NaN,"nm0450407, nm0485392"
3,tt0038687,Let There Be Light,1980,58,"Documentary,War",7.4,nm0404158,NaN,NaN,NaN,nm0001379,NaN,NaN,NaN,nm0442105
4,tt0039442,"Habla, mudita",1973,88,Drama,6.1,"nm0007023, nm0019330","nm0544330, nm0370455",nm0190834,nm0006280,nm0349426,nm0215327,NaN,nm0651204,nm0305869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326307,tt9916188,Minotaur,\N,\N,Thriller,NaN,NaN,NaN,NaN,NaN,nm2410311,NaN,nm0865189,NaN,NaN
326308,tt9916190,Safeguard,2020,95,"Action,Adventure,Thriller",3.7,"nm0302466, nm5464699, nm2018573, nm2009056",NaN,"nm8262250, nm5785503",nm7879007,nm7308376,nm4877791,nm10299811,NaN,NaN
326309,tt9916270,Il talento del calabrone,2020,84,Thriller,5.8,"nm0144812, nm3080119","nm2063290, nm9428255",nm0130846,nm2747888,nm1480867,NaN,"nm1799384, nm0656465",NaN,nm10538402
326310,tt9916362,Coven,2020,92,"Drama,History",6.4,"nm0107165, nm0266723","nm3766704, nm10678594",NaN,nm5813626,nm1893148,NaN,"nm2970042, nm4065853, nm1086949",NaN,nm3471432


In [ ]:
#export cleaned data to csv

film_data.to_csv('webpage/film_data.csv')